In [47]:
import pulp

# Simple Problem

In [48]:
my_lp_problem = pulp.LpProblem("My LP Problem", pulp.LpMaximize)

In [49]:
x = pulp.LpVariable('x', lowBound=0, cat='Continuous')
y = pulp.LpVariable('y', lowBound=2, cat='Continuous')

In [50]:
# Objective function
my_lp_problem += 4 * x + 3 * y, "Z"

In [51]:
# Constraints
my_lp_problem += 2 * y <= 25 - x
my_lp_problem += 4 * y >= 2 * x - 8
my_lp_problem += y <= 2 * x - 5

In [52]:
my_lp_problem

My LP Problem:
MAXIMIZE
4*x + 3*y + 0
SUBJECT TO
_C1: x + 2 y <= 25

_C2: - 2 x + 4 y >= -8

_C3: - 2 x + y <= -5

VARIABLES
x Continuous
2 <= y Continuous

In [53]:
my_lp_problem.solve()

1

In [54]:
my_lp_problem.variables()

[x, y]

In [55]:
pulp.value(my_lp_problem.objective)

73.75

# Resourcing Problem

In [56]:
car_problem = pulp.LpProblem("Car Problem", pulp.LpMaximize)

In [57]:
A = pulp.LpVariable('A', lowBound=0, cat='Integer')
B = pulp.LpVariable('B', lowBound=0, cat='Integer')

In [58]:
# Objective function
car_problem += 30000 * A + 45000 * B, "C"

In [59]:
car_problem

Car Problem:
MAXIMIZE
30000*A + 45000*B + 0
VARIABLES
0 <= A Integer
0 <= B Integer

In [60]:
# Constraints
car_problem += 3*A + 4*B  <= 30
car_problem += 5*A + 6*B  <= 60
car_problem += 1.5*A + 3*B  <= 21

In [61]:
car_problem

Car Problem:
MAXIMIZE
30000*A + 45000*B + 0
SUBJECT TO
_C1: 3 A + 4 B <= 30

_C2: 5 A + 6 B <= 60

_C3: 1.5 A + 3 B <= 21

VARIABLES
0 <= A Integer
0 <= B Integer

In [62]:
car_problem.solve()

1

In [63]:
for variable in car_problem.variables():
    print("{} = {}".format(variable.name, variable.varValue))

A = 2.0
B = 6.0


In [64]:
pulp.value(car_problem.objective)

330000.0

# Blending Problem

In [65]:
model = pulp.LpProblem("Sausage Problem", pulp.LpMinimize)

In [66]:
sausage_types = ['economy', 'premium']
ingredients = ['pork', 'wheat', 'starch']

In [67]:
ing_weight = pulp.LpVariable.dicts("weight kg",
                                     ((i, j) for i in sausage_types for j in ingredients),
                                     lowBound=0,
                                     cat='Continuous')

In [68]:
model += (
    pulp.lpSum([
        4.32 * ing_weight[(i, 'pork')]
        + 2.46 * ing_weight[(i, 'wheat')]
        + 1.86 * ing_weight[(i, 'starch')]
        for i in sausage_types])
)

In [69]:
model += pulp.lpSum([ing_weight['economy', j] for j in ingredients]) == 350 * 0.05
model += pulp.lpSum([ing_weight['premium', j] for j in ingredients]) == 500 * 0.05

# Economy has >= 40% pork, premium >= 60% pork
model += ing_weight['economy', 'pork'] >= (
    0.4 * pulp.lpSum([ing_weight['economy', j] for j in ingredients]))

model += ing_weight['premium', 'pork'] >= (
    0.6 * pulp.lpSum([ing_weight['premium', j] for j in ingredients]))

# Sausages must be <= 25% starch
model += ing_weight['economy', 'starch'] <= (
    0.25 * pulp.lpSum([ing_weight['economy', j] for j in ingredients]))

model += ing_weight['premium', 'starch'] <= (
    0.25 * pulp.lpSum([ing_weight['premium', j] for j in ingredients]))

# We have at most 30 kg of pork, 20 kg of wheat and 17 kg of starch available
model += pulp.lpSum([ing_weight[i, 'pork'] for i in sausage_types]) <= 30
model += pulp.lpSum([ing_weight[i, 'wheat'] for i in sausage_types]) <= 20
model += pulp.lpSum([ing_weight[i, 'starch'] for i in sausage_types]) <= 17

# We have at least 23 kg of pork to use up
model += pulp.lpSum([ing_weight[i, 'pork'] for i in sausage_types]) >= 23

In [70]:
model.solve()
pulp.LpStatus[model.status]

'Optimal'

In [71]:
for var in ing_weight:
    var_value = ing_weight[var].varValue
    print ("The weight of {0} in {1} sausages is {2:.2f} kg".format(var[1], var[0], var_value))

The weight of pork in economy sausages is 7.00 kg
The weight of wheat in economy sausages is 6.12 kg
The weight of starch in economy sausages is 4.38 kg
The weight of pork in premium sausages is 16.00 kg
The weight of wheat in premium sausages is 2.75 kg
The weight of starch in premium sausages is 6.25 kg


In [72]:
total_cost = pulp.value(model.objective)

print( "The total cost is €{} for 350 economy sausages and 500 premium sausages".format(round(total_cost, 2)))

The total cost is €140.96 for 350 economy sausages and 500 premium sausages


# Factory Problem

In [73]:
import pandas as pd

In [74]:
factories = pd.read_csv('factory_variables.csv', index_col=['Month','Factory'])

In [75]:
demand = pd.read_csv('https://raw.githubusercontent.com/benalexkeen/Introduction-to-linear-programming/master/csv/monthly_demand.csv', index_col=['Month'])

In [76]:
production = pulp.LpVariable.dicts("production",
                                     ((month, factory) for month, factory in factories.index),
                                     lowBound=0,
                                     cat='Integer')

In [77]:
factory_status = pulp.LpVariable.dicts("factory_status",
                                     ((month, factory) for month, factory in factories.index),
                                     cat='Binary')

In [78]:
model = pulp.LpProblem("Cost minimising scheduling problem", pulp.LpMinimize)

In [79]:
model += pulp.lpSum(
    [production[month, factory] * factories.loc[(month, factory), 'Variable_Costs'] for month, factory in factories.index]
    + [factory_status[month, factory] * factories.loc[(month, factory), 'Fixed_Costs'] for month, factory in factories.index]
)

In [80]:
months = demand.index
for month in months:
    model += production[(month, 'A')] + production[(month, 'B')] == demand.loc[month, 'Demand']

In [81]:
for month, factory in factories.index:
    min_production = factories.loc[(month, factory), 'Min_Capacity']
    max_production = factories.loc[(month, factory), 'Max_Capacity']
    model += production[(month, factory)] >= min_production * factory_status[month, factory]
    model += production[(month, factory)] <= max_production * factory_status[month, factory]

In [82]:
model += factory_status[5, 'B'] == 0
model += production[5, 'B'] == 0

In [83]:
model.solve()
pulp.LpStatus[model.status]

'Optimal'

# Toy

In [87]:
model_toy = pulp.LpProblem('Toy', pulp.LpMaximize)

In [88]:
A = pulp.LpVariable('A', lowBound=0, cat='Integer')
B = pulp.LpVariable('B', lowBound=0, cat='Integer')

In [89]:
model_toy += 2*A+3*B, 'Z'

In [90]:
model_toy

Toy:
MAXIMIZE
2*A + 3*B + 0
VARIABLES
0 <= A Integer
0 <= B Integer

In [92]:
model_toy += 8*A+14*B<=20000
model_toy += A+B<=2000

In [93]:
model_toy.solve()
pulp.LpStatus[model_toy.status]

'Optimal'

In [98]:
for var in model_toy.variables():
#     var_value = model_toy[var].varValue
    print (var.name, var.varValue)

A 1334.0
B 666.0


In [101]:
pulp.value(model_toy.objective)

4666.0

# Table

In [112]:
model_table = pulp.LpProblem('Table', pulp.LpMaximize)

In [113]:
T1 = pulp.LpVariable('T1', lowBound=0, cat='Integer')
T2 = pulp.LpVariable('T2', lowBound=0, cat='Integer')

In [114]:
model_table += 90*T1+110*T2, 'Z'

In [115]:
model_table

Table:
MAXIMIZE
90*T1 + 110*T2 + 0
VARIABLES
0 <= T1 Integer
0 <= T2 Integer

In [116]:
model_table += 2*T1+4*T2<=7000
model_table += T1+2.5*T2<=4000
model_table += 2*T1+1.5*T2<=5500

In [117]:
model_table.solve()
pulp.LpStatus[model_table.status]

'Optimal'

In [119]:
for var in model_table.variables():
#     var_value = model_toy[var].varValue
    print (var.name, var.varValue)

T1 2300.0
T2 600.0


In [120]:
pulp.value(model_table.objective)

273000.0